In [1]:
### Convolutional Neural Network (CNN) Model Building (Training and Testing)

import tensorflow as tf

### this is the path of the data and labels
### for both training data set and testing
### data set 

path_train_img = '../data/training/img'
train_label = '../data/training/label/labels.txt'

### read the label txt file and unpack label
### information about each individual image 
### return a dictionary with image name and 
### label

def unpack_labels(label_path):
    labels = list()
    dict_ = dict()
    with open(label_path, 'r') as f:
        labels = f.readlines()
    
    for label in labels:
        img = label.split(':')[0]
        lbl = label.split(':')[1][2:][:-2]
        dict_[img] = lbl
        
    return dict_

train_label_dict = unpack_labels(train_label)

In [2]:
### for later model building
### one hot encoding 

def one_hot_encoding(img, dict_):
    import numpy as np

    if dict_[img] == 'airplane':
        ohl = np.array([1,0,0,0,0,0,0,0,0,0])
    elif dict_[img] == 'automobile':
        ohl = np.array([0,1,0,0,0,0,0,0,0,0])
    elif dict_[img] == 'bird':
        ohl = np.array([0,0,1,0,0,0,0,0,0,0])
    elif dict_[img] == 'cat':
        ohl = np.array([0,0,0,1,0,0,0,0,0,0])
    elif dict_[img] == 'deer':
        ohl = np.array([0,0,0,0,1,0,0,0,0,0])
    elif dict_[img] == 'dog':
        ohl = np.array([0,0,0,0,0,1,0,0,0,0])
    elif dict_[img] == 'frog':
        ohl = np.array([0,0,0,0,0,0,1,0,0,0])
    elif dict_[img] == 'horse':
        ohl = np.array([0,0,0,0,0,0,0,1,0,0])
    elif dict_[img] == 'ship':
        ohl = np.array([0,0,0,0,0,0,0,0,1,0])
    else:
        ohl = np.array([0,0,0,0,0,0,0,0,0,1])
        
    return ohl

In [3]:
def data_with_label(data_path, dict_):
    from tqdm import tqdm
    import os
    import numpy as np
    import cv2
    from random import shuffle
    
    imgs = []
    
    for i in tqdm(os.listdir(data_path)):
        path = os.path.join(data_path, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (32,32))
        imgs.append([np.array(img), one_hot_encoding(i, dict_)])
        
    shuffle(imgs)
    return imgs

In [4]:
train_data_with_label = data_with_label(path_train_img, train_label_dict)

100%|██████████| 50000/50000 [01:15<00:00, 662.09it/s]


In [5]:
def unzip_data(data_with_label):
    import numpy as np
    img_data_flat = np.array([d[0] for d in data_with_label]).reshape(-1, 32, 32, 1)
    lbl_data_flat = np.array([d[1] for d in data_with_label])
    return img_data_flat, lbl_data_flat

In [6]:
train_img_flat, train_lbl_flat = unzip_data(train_data_with_label)

In [8]:
### now we can start to build the machine learning model
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

def create_model():
    model = Sequential()
    model.add(InputLayer(input_shape = [32,32,1]))
    model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=5, padding='same'))

    model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=5, padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(10, activation='softmax'))
    optimizer = Adam(lr=1e-3)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

Using TensorFlow backend.


In [ ]:
### train the model
print('training... approx. for an hour. Be patient')
model = create_model()
model.fit(x=train_img_flat, y=train_lbl_flat, epochs=50, batch_size=100, verbose=0)

training... approx. for an hour. Be patient


In [ ]:
### evaluate the model based on training data 
scores = model.evaluate(train_img_flat, train_lbl_flat, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
### save the weights file
### serialize weights to HDF5
model.save_weights("../data/model/model.h5")
print("Saved model to disk")